In [69]:
import nltk
import os
from nltk import word_tokenize,sent_tokenize,pos_tag
from nltk.tag import StanfordPOSTagger
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
lemmatizer=WordNetLemmatizer()
punctuations = '''!()-[]{};—:'"\,<>./’?@#$%^&*_~“”'''

In [70]:
senti_words = []
f=open("improvedsamplelexicon.txt","r",encoding="utf-8") 
all_lines=f.readlines()
for line in all_lines:
    
    w=line.split('\t')[1]
    senti_words.append(w)

In [71]:
stopwords=['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves','he','him','his','himself'
,'she','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom',
 'this','that','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a',
 'an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','between','into','through','during',
 'before','after','to','from','again','then','once','here','there','when','where','why','how','all','any','both','each',
 'than','should','d','ll','m','o','re','ve','y','ma']

In [72]:
stopwords=stopwords+list(punctuations)

In [73]:
neg= {"aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never","no" "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite","unfulfilled","undue","anti"}

In [74]:
boost={"absolutely":0.5, "amazingly":0.125, "awfully":0.25, "completely":0.25, "considerably":0.125,
 "decidedly":0.25 , "deeply":0.25 , "enormously":0.25 ,
 "entirely":0.5 , "especially":0.25 , "exceptionally":0.25, "extremely":0.625 ,"fabulously":0.25   , "fully":0.375  ,
 "greatly":0.125 , "highly":0.5   ,"huge":0.25, "hugely":0.25   , "incredibly":0.25   ,
 "intensely":0.25   , "majorly":0.625  ,"particularly":0.125 ,"really":0.375 ,"substantially":0.125   ,
 "thoroughly":0.625   , "totally":0.5  ,"unbelievably":0.25   , "utterly":0.5   ,
 "very":0.25  ,"tremendously":-0.25 ,  "barely":-0.375   ,"badly":-0.125 ,"hardly":-0.25,"unusually":-0.5   , 
 "less":-0.5   , "little":-0.375   , "marginally":-0.125   , "partly":-0.125   ,
 "scarcely":-0.25   , "slightly":-0.25   , "somewhat":-0.125, "shoddy":-0.625, "poorly":-0.75}
pos_boost=["absolutely", "amazingly", "awfully", "completely", "considerably",
 "decidedly","deeply","enormously","entirely","especially","exceptionally","extremely","fabulously","fully",
 "greatly","highly","huge","hugely", "incredibly","intensely","majorly","particularly","really","substantially","thoroughly","totally","unbelievably", "utterly","very"]
neg_boost=["tremendously", "barely","badly","hardly","unusually", "less", "little", "marginally", "partly",
 "scarcely", "slightly","somewhat", "shoddy", "poorly"]

In [75]:
def convert_tag(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [76]:
def find_sentiment(word,pos):
    found=0
    line_nos=[index for index, value in enumerate(senti_words) if value == word]
    lines=[all_lines[lno] for lno in line_nos]
    if len(lines)==0:
        return None
    for l in lines:
        if l[0]==pos:
            pos_score,neg_score=l.split('\t')[2],l.split('\t')[3]
            found=1
            break
        else:
            continue
    if found==1:
        return(float(pos_score)-float(neg_score))
    else:
        pos_score,neg_score=lines[0].split('\t')[2],lines[0].split('\t')[3]
        return(float(pos_score)-float(neg_score))

In [77]:
def get_sentiment(word,pos):
    second_sent=find_sentiment(word,pos)
            
    if second_sent is None:
        try:
            lemma=lemmatizer.lemmatize(word,pos)
            synsets = wn.synsets(lemma)
            synset=synsets[0]
            s=synset.name()
            i=s.index('.')
            second_sent=find_sentiment(s[:i],pos)
            #print(s[:i],second_sent)
            if second_sent is None:
                second_sent=0.0
        except:
            second_sent=0.0
    return second_sent

In [78]:
def normalize(score):
    if score>=0.0:
        return((score-(0.0))/1.5)
    else:
        return(-(-score-(0.03582397526189089))/1.839176024738109)

In [79]:
nouns=open('sent_nouns.txt','r',encoding='utf-8')
nouns=nouns.read()
nouns=nouns.split()

In [80]:
def clean_text(t):
    for c in stopwords:
        try:
             t.remove(c)
        except:
            continue
    return t

In [81]:
def positive_booster(booster,word,pos2):
    sec_score=get_sentiment(word,pos2)
    if sec_score>0.0:
        return(boost[booster]+sec_score)
    if sec_score<0.0:
        return(sec_score-boost[booster])
    else:
        #return(boost[booster])
        return 0.0

In [82]:
def negative_booster(booster,word,pos2):
    sec_score=get_sentiment(word,pos2)
    if sec_score>0.0:
        return(sec_score+boost[booster])
    if sec_score<0.0:
        return(sec_score+boost[booster])
    else:
        #return(boost[booster])
        return 0.0

In [83]:
def check(bigrams):
    score=0.0
    for b in bigrams:
        #b_tag=st.tag(b)
        b_tag=pos_tag(b)
        #print(b_tag)
        second_gram=b_tag[1]
        first_gram=b_tag[0]
        if second_gram[0] in nouns or second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VBG') or second_gram[1].startswith('VBN') or second_gram[1].startswith('VBD'):
            pos2=convert_tag(second_gram[1])
            pos1=convert_tag(first_gram[1])
        
            if first_gram[0] in boost:
                if first_gram[0] in pos_boost:
                    score=positive_booster(first_gram[0],second_gram[0],pos2)
                    print('Pos-Boost:',b_tag,score)
                if first_gram[0] in neg_boost:
                    score=negative_booster(first_gram[0],second_gram[0],pos2)
                    print('Neg-Boost:',b_tag,score)
            
   
            if first_gram[0] in neg:
                x=0.0
                sec_score=get_sentiment(second_gram[0],pos2)
                print(second_gram[0],sec_score)
                if sec_score ==0.0 or sec_score==None:
                    try:
                        x=get_sentiment(first_gram[0],pos1)
                        score+=(x+sec_score)
                    except:
                        print(x)
                else:
                    score+=-(sec_score)
                #print('Neg:',b_tag,score)
            
            elif first_gram[0] not in neg and first_gram[0] not in boost:
                try:
                    first_score=get_sentiment(first_gram[0],pos1)
                    score+=first_score+get_sentiment(second_gram[0],pos2)
                    #print('Normal:',b_tag,score)
                except:
                    print("Not Found:",second_gram[0])
                    #not_found.add(second_gram[0])
    return ((score))
    
    

In [84]:
path="D:/LEX/ProjectS-master/DATA/All_July30/"
l=os.listdir(path)
rw=open("news_result.txt","a",encoding='utf-8')

score_list=[]
for i in l:
    #try:
        file=open(path+i,"r",encoding='utf-8')
        article=file.read()
        score=0.0
        tok_tweet=(clean_text(word_tokenize(article.lower())))
        bigrams=nltk.ngrams(tok_tweet,2)
        score=check(bigrams)
        print(score)
        score_list.append(score)
        file.close()
    #except:
        print(i)
rw.close()

Pos-Boost: [('fully', 'RB'), ('geared', 'VBN')] 0.0
Pos-Boost: [('absolutely', 'RB'), ('ready', 'JJ')] 1.0
air-conditioned 0.0
Pos-Boost: [('fully', 'RB'), ('geared', 'VBN')] 0.0
Pos-Boost: [('absolutely', 'RB'), ('ready', 'JJ')] 1.0
Neg-Boost: [('badly', 'RB'), ('hit', 'VBD')] -0.25
seen 0.0
Pos-Boost: [('very', 'RB'), ('clear', 'JJ')] 0.75
Pos-Boost: [('fully', 'RB'), ('geared', 'VBN')] 0.0
Pos-Boost: [('absolutely', 'RB'), ('ready', 'JJ')] 1.0
expecting 0.0
Pos-Boost: [('huge', 'JJ'), ('concern', 'NN')] -0.875
looked 0.0
Pos-Boost: [('fully', 'RB'), ('defined', 'VBN')] 0.75
issued 0.0
full 0.0
rising 0.5
possible 0.5
expecting 0.0
changed 0.0
Neg-Boost: [('marginally', 'RB'), ('cheaper', 'JJR')] 0.375
more 0.0
same 0.0
more 0.0
Pos-Boost: [('fully', 'RB'), ('geared', 'VBN')] 0.0
0.75
01.txt
sure 0.25
Pos-Boost: [('completely', 'RB'), ('built', 'VBN')] 0.0
taking 0.0
procuring 0.125
reaping 0.0
Pos-Boost: [('fully', 'RB'), ('geared', 'VBN')] 0.0
sure 0.25
Pos-Boost: [('fully', 'RB'),

Pos-Boost: [('fully', 'RB'), ('exempt', 'JJ')] -0.875
Pos-Boost: [('particularly', 'RB'), ('problematic', 'JJ')] -0.625
Pos-Boost: [('completely', 'RB'), ('unsubstantiated', 'JJ')] 0.0
available 0.375
Pos-Boost: [('fully', 'RB'), ('exempt', 'JJ')] -0.875
true 0.125
Pos-Boost: [('fully', 'RB'), ('exempt', 'JJ')] -0.875
Pos-Boost: [('fully', 'RB'), ('exempt', 'JJ')] -0.875
creating 0.0
ready 0.5
Pos-Boost: [('really', 'RB'), ('good', 'JJ')] 1.0
Pos-Boost: [('really', 'RB'), ('interesting', 'JJ')] 0.75
Pos-Boost: [('very', 'RB'), ('positive', 'JJ')] 1.0
Pos-Boost: [('very', 'RB'), ('important', 'JJ')] 0.5
Neg-Boost: [('slightly', 'RB'), ('better', 'RBR')] 0.5
wish 0.125
Pos-Boost: [('particularly', 'RB'), ('true', 'JJ')] 0.25
practical 0.0
complete 0.0
Pos-Boost: [('really', 'RB'), ('robust', 'JJ')] 0.75
bad -0.625
bad -0.625
expected 0.0
Pos-Boost: [('really', 'RB'), ('looking', 'VBG')] 0.0
seen 0.0
Pos-Boost: [('really', 'RB'), ('translating', 'VBG')] 0.0
happening 0.0
Pos-Boost: [('rea

In [86]:
max(score_list)

17.007651182768576